## Finalize the single cell perturbseq dataset

The output of the Seurat pipeline in 2.process-perturbseq.ipynb is not easily compatible with our downstream tasks.
This notebook finalizes the input perturbseq (CRISPRi) dataset.

There are four basic steps:

1. Load, transpose, and clean the gene expression matrix
2. Load the single cell file identities (barcodes and metadata)
3. Merge
4. Output

Lastly, we use pycytominer.aggregate to form bulk profiles from the single cell readouts

In [1]:
import pathlib
import numpy as np
import pandas as pd

from pycytominer import aggregate

In [2]:
gse_id = "GSE132080"
perturbseq_data_dir = pathlib.Path("data/perturbseq/")

output_file = pathlib.Path(f"{perturbseq_data_dir}/{gse_id}_final_analytical.tsv.gz")
output_bulk_file = pathlib.Path(f"{perturbseq_data_dir}/{gse_id}_bulk_final_analytical.tsv.gz")

In [3]:
# Load and process gene expression data
gene_exp_file = pathlib.Path(f"{perturbseq_data_dir}/{gse_id}_processed_matrix.tsv.gz")

gene_exp_df = (
    pd.read_csv(gene_exp_file, sep="\t", index_col=0)
    .transpose()
    .reset_index()
    .rename({"index": "Metadata_barcode"}, axis="columns")
)

# Pull out the measured genes
gene_features = gene_exp_df.columns.tolist()
gene_features.remove("Metadata_barcode")

gene_exp_df = gene_exp_df.assign(Metadata_sequence=[x.split("-")[0] for x in gene_exp_df.Metadata_barcode])
gene_exp_df.columns.name = ""

meta_features = ["Metadata_barcode", "Metadata_sequence"]
gene_features = sorted(gene_exp_df.drop(meta_features, axis="columns").columns.tolist())

gene_exp_df = gene_exp_df.reindex(meta_features + gene_features, axis="columns")

print(gene_exp_df.shape)
gene_exp_df.head()

(23537, 1002)


,Metadata_barcode,Metadata_sequence,ABCA1,ABCC3,ABI3BP,AC002331.1,AC002480.3,AC003092.1,AC005616.2,AC006262.5,...,YPEL4,YPEL5,ZBTB38,ZFAS1,ZFP36L1,ZNF365,ZNF43,ZNF483,ZNF556,ZYX
0,AAACCTGAGAGTAATC-1,AAACCTGAGAGTAATC,-0.078689,-0.093494,-0.01406,-0.013653,-0.018973,-0.026806,-0.034066,-0.485621,...,-0.09309,-0.632468,-0.539091,-1.633805,-0.876912,-0.012155,-0.017763,-0.026877,-0.0524,-0.452229
1,AAACCTGAGGGATCTG-1,AAACCTGAGGGATCTG,-0.078689,-0.093494,-0.01406,-0.013653,-0.018973,-0.026806,-0.034066,-0.485621,...,-0.09309,-0.632468,-0.539091,1.610220,1.322265,-0.012155,-0.017763,-0.026877,-0.0524,-0.452229
2,AAACCTGAGGTCATCT-1,AAACCTGAGGTCATCT,-0.078689,-0.093494,-0.01406,-0.013653,-0.018973,-0.026806,-0.034066,-0.485621,...,-0.09309,-0.632468,-0.539091,-0.567252,0.057540,-0.012155,-0.017763,-0.026877,-0.0524,-0.452229
3,AAACCTGCAATGGAGC-1,AAACCTGCAATGGAGC,-0.078689,-0.093494,-0.01406,-0.013653,-0.018973,-0.026806,-0.034066,2.237599,...,-0.09309,1.765279,-0.539091,-1.659420,-0.876912,-0.012155,-0.017763,-0.026877,-0.0524,-0.452229
4,AAACCTGCACCAGGCT-1,AAACCTGCACCAGGCT,-0.078689,-0.093494,-0.01406,-0.013653,-0.018973,-0.026806,-0.034066,-0.485621,...,-0.09309,0.686753,-0.539091,0.541260,0.934539,-0.012155,-0.017763,-0.026877,-0.0524,1.406718


In [4]:
# Load cell identities
identity_file = pathlib.Path(f"{perturbseq_data_dir}/{gse_id}_cell_identities.csv.gz")
cell_id_df = pd.read_csv(identity_file, sep=",")

cell_id_df.columns = [f"Metadata_{x}" for x in cell_id_df.columns]
cell_id_df = cell_id_df.assign(Metadata_gene_identity=[str(x).split("_")[0] for x in cell_id_df.Metadata_guide_identity])

print(cell_id_df.shape)
cell_id_df.head()

(23608, 9)


,Metadata_cell_barcode,Metadata_guide_identity,Metadata_read_count,Metadata_UMI_count,Metadata_coverage,Metadata_gemgroup,Metadata_good_coverage,Metadata_number_of_cells,Metadata_gene_identity
0,GGACAAGTCCCTGACT-3,neg_ctrl_non-targeting_00028,7452,457,16.306346,3,True,1,neg
1,CGACTTCAGAAGGCCT-3,GNB2L1_GNB2L1_+_180670873.23-P1P2_13,6554,361,18.155125,3,True,1,GNB2L1
2,TTAGGCAAGAAGGCCT-2,TUBB_TUBB_+_30688126.23-P1_00,4177,165,25.315152,2,True,2,TUBB
3,CGTAGGCAGCCAGGAT-1,TUBB_TUBB_+_30688126.23-P1_01,4024,218,18.458716,1,True,1,TUBB
4,GCGCAACTCACGATGT-2,HSPE1_HSPE1_+_198365089.23-P1P2_00,3923,134,29.276119,2,True,1,HSPE1


In [5]:
# Merge single cells with identifiers
sc_df = cell_id_df.merge(
    gene_exp_df,
    how="right",
    right_on="Metadata_barcode",
    left_on="Metadata_cell_barcode"
)

sc_df = sc_df.reset_index().rename({"index": "Metadata_cell_identity"}, axis="columns")
sc_df.Metadata_cell_identity = [f"sc_profile_{x}" for x in sc_df.Metadata_cell_identity]

print(sc_df.shape)
sc_df.head()

(23537, 1012)


,Metadata_cell_identity,Metadata_cell_barcode,Metadata_guide_identity,Metadata_read_count,Metadata_UMI_count,Metadata_coverage,Metadata_gemgroup,Metadata_good_coverage,Metadata_number_of_cells,Metadata_gene_identity,...,YPEL4,YPEL5,ZBTB38,ZFAS1,ZFP36L1,ZNF365,ZNF43,ZNF483,ZNF556,ZYX
0,sc_profile_0,AAACCTGAGAGTAATC-1,RAN_RAN_+_131356438.23-P1P2_12,544.0,34.0,16.000000,1.0,True,1.0,RAN,...,-0.09309,-0.632468,-0.539091,-1.633805,-0.876912,-0.012155,-0.017763,-0.026877,-0.0524,-0.452229
1,sc_profile_1,AAACCTGAGGGATCTG-1,neg_ctrl_non-targeting_00089,267.0,19.0,14.052632,1.0,True,1.0,neg,...,-0.09309,-0.632468,-0.539091,1.610220,1.322265,-0.012155,-0.017763,-0.026877,-0.0524,-0.452229
2,sc_profile_2,AAACCTGAGGTCATCT-1,POLR2H_POLR2H_+_184081251.23-P1P2_08,622.0,34.0,18.294118,1.0,True,1.0,POLR2H,...,-0.09309,-0.632468,-0.539091,-0.567252,0.057540,-0.012155,-0.017763,-0.026877,-0.0524,-0.452229
3,sc_profile_3,AAACCTGCAATGGAGC-1,TUBB_TUBB_+_30688126.23-P1_03,433.0,20.0,21.650000,1.0,True,1.0,TUBB,...,-0.09309,1.765279,-0.539091,-1.659420,-0.876912,-0.012155,-0.017763,-0.026877,-0.0524,-0.452229
4,sc_profile_4,AAACCTGCACCAGGCT-1,CDC23_CDC23_-_137548987.23-P1P2_04,136.0,8.0,17.000000,1.0,True,1.0,CDC23,...,-0.09309,0.686753,-0.539091,0.541260,0.934539,-0.012155,-0.017763,-0.026877,-0.0524,1.406718


In [6]:
# Write the file to disk
sc_df.to_csv(
    output_file,
    index=False,
    sep="\t",
    compression={"method": "gzip", "mtime": 1}
)

## Calculate bulk perturbseq data

In [7]:
# Perform single cell aggregation into bulk
bulk_df = aggregate(
    population_df=sc_df,
    strata=["Metadata_guide_identity"],
    features=gene_features,
    operation="median"
)

# remove one row with NaN value
bulk_df = bulk_df[~bulk_df['Metadata_guide_identity'].isnull()]

# create a column for the gene
bulk_df = (
    bulk_df
    .assign(Metadata_gene_identity=[x.split("_")[0] for x in bulk_df.Metadata_guide_identity])
    .query("Metadata_gene_identity != '*'")
)

bulk_df = bulk_df.reindex(["Metadata_guide_identity", "Metadata_gene_identity"] + gene_features, axis="columns")

print(bulk_df.shape)
bulk_df.head()

(138, 1002)


,Metadata_guide_identity,Metadata_gene_identity,ABCA1,ABCC3,ABI3BP,AC002331.1,AC002480.3,AC003092.1,AC005616.2,AC006262.5,...,YPEL4,YPEL5,ZBTB38,ZFAS1,ZFP36L1,ZNF365,ZNF43,ZNF483,ZNF556,ZYX
1,ALDOA_ALDOA_+_30077139.23-P1P2_00,ALDOA,-0.078689,-0.093494,-0.01406,-0.013653,-0.018973,-0.026806,-0.034066,-0.485621,...,-0.09309,-0.632468,-0.539091,-0.170790,0.014776,-0.012155,-0.017763,-0.026877,-0.0524,-0.452229
2,ALDOA_ALDOA_+_30077139.23-P1P2_06,ALDOA,-0.078689,-0.093494,-0.01406,-0.013653,-0.018973,-0.026806,-0.034066,-0.485621,...,-0.09309,-0.632468,-0.539091,-0.045114,0.042050,-0.012155,-0.017763,-0.026877,-0.0524,-0.452229
3,ALDOA_ALDOA_+_30077139.23-P1P2_07,ALDOA,-0.078689,-0.093494,-0.01406,-0.013653,-0.018973,-0.026806,-0.034066,-0.485621,...,-0.09309,-0.632468,-0.539091,-0.101183,0.027887,-0.012155,-0.017763,-0.026877,-0.0524,-0.452229
4,ALDOA_ALDOA_+_30077139.23-P1P2_13,ALDOA,-0.078689,-0.093494,-0.01406,-0.013653,-0.018973,-0.026806,-0.034066,-0.485621,...,-0.09309,-0.632468,-0.539091,-0.050848,-0.107389,-0.012155,-0.017763,-0.026877,-0.0524,-0.452229
5,ALDOA_ALDOA_+_30077139.23-P1P2_14,ALDOA,-0.078689,-0.093494,-0.01406,-0.013653,-0.018973,-0.026806,-0.034066,-0.485621,...,-0.09309,-0.632468,-0.539091,0.150701,0.051095,-0.012155,-0.017763,-0.026877,-0.0524,-0.452229


In [8]:
# Write the file to disk
bulk_df.to_csv(
    output_bulk_file,
    index=False,
    sep="\t",
    compression={"method": "gzip", "mtime": 1}
)